In [56]:
import tensorflow as tf
import cv2
import numpy as np
import os
from tqdm import tqdm

In [79]:
def load_model():
    print("Loading the model...")
    global model
    model = tf.keras.models.load_model("models/handwash_model_v1.2.keras")
    # model = tf.keras.models.load_model("models/downloaded_model.keras")
    print("model loaded successfully!")

load_model()

Loading the model...
model loaded successfully!


In [72]:
def predict(vid_pth):
    vidcap = cv2.VideoCapture(vid_pth)

    is_success, image = vidcap.read()
    frame_number = 0

    freq_dict = dict()

    while is_success:
        if frame_number%20 == 0:
            img = image[..., ::-1] / 255.
            img = np.expand_dims(cv2.resize(img, (256, 256)), axis=0)
            probas = model.predict(img, verbose=0)
            pred = probas.argmax()
            if pred in freq_dict:
                freq_dict[pred] += 1
            else:
                freq_dict[pred] = 1
        is_success, image = vidcap.read()
        frame_number += 1

    labels_dict = {0: "Rub both wrists in rotating manner",
                1: "Rub your palms together",
                2: "Rub the back of your fingers and hands",
                3: "Rub your hands by interlocking your fingers",
                4: "Interlock fingers and rub the back of fingers of both hands",
                5: "Rub the area between index finger and thumb",
                6: "Rub fingertips on palm of both hands in circular manner"}

    max_freq_label = max(freq_dict.items(), key=lambda x: x[1])[0]

    # return f"{labels_dict[max_freq_label]}"
    return f"{max_freq_label}"

In [73]:
predict("dataset/videos/test/0/HandWash_002_A_12_G_01.mp4")

'0'

In [54]:
incorrect_preds = []
test_videos_dir = "/home/vikas/Projects/freelancing/research/dataset/videos/test"
for folder in os.listdir(test_videos_dir):
    for file in os.listdir(f"{test_videos_dir}/{folder}"):
        if folder != predict(f"{test_videos_dir}/{folder}/{file}"):
            incorrect_preds.append(f"{test_videos_dir}/{folder}/{file}")

In [55]:
len(incorrect_preds)

22

In [23]:
total_test_videos = 0
test_videos_dir = "/home/vikas/Projects/freelancing/research/dataset/videos/test"
for folder in os.listdir(test_videos_dir):
    total_test_videos += len(os.listdir(f"{test_videos_dir}/{folder}"))

In [24]:
total_test_videos

82

In [46]:
incorrect_preds_dict = {}
for path in incorrect_preds:
    incorrect_label = path.split("/")[-2]
    if incorrect_label in incorrect_preds_dict:
        incorrect_preds_dict[incorrect_label].append(path)
    else:
        incorrect_preds_dict[incorrect_label] = [path]

In [47]:
incorrect_preds_dict["0"]

['/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_015_A_12_G_04.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_018_A_11_G_05.mp4']

In [48]:
incorrect_preds

['/home/vikas/Projects/freelancing/research/dataset/videos/test/4/HandWash_019_A_05_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/4/HandWash_018_A_06_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/4/HandWash_019_A_06_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/4/HandWash_020_A_06_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/4/HandWash_020_A_05_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_015_A_12_G_04.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_018_A_11_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/3/HandWash_024_A_04_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/3/HandWash_021_A_04_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/5/HandWash_024_A_07_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/datase

In [49]:
len(incorrect_preds_dict["0"])

2

In [79]:
incorrect_preds_dict["6"]

['/home/vikas/Projects/freelancing/research/dataset/videos/test/6/HandWash_007_A_09_G_01.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/6/HandWash_021_A_10_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/6/HandWash_006_A_09_G_01.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/6/HandWash_008_A_09_G_01.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/6/HandWash_018_A_09_G_05.mp4']

In [71]:
labels_dict = {0: "Rub both wrists in rotating manner",
            1: "Rub your palms together",
            2: "Rub the back of your fingers and hands",
            3: "Rub your hands by interlocking your fingers",
            4: "Interlock fingers and rub the back of fingers of both hands",
            5: "Rub the area between index finger and thumb",
            6: "Rub fingertips on palm of both hands in circular manner"}

In [78]:
labels_dict[6]

'Rub fingertips on palm of both hands in circular manner'

In [84]:
for k, v in incorrect_preds_dict.items():
    print(f"label {k} has {len(v)} incorrect predictions")

label 4 has 5 incorrect predictions
label 0 has 2 incorrect predictions
label 3 has 2 incorrect predictions
label 5 has 3 incorrect predictions
label 6 has 5 incorrect predictions
label 2 has 1 incorrect predictions


### Predict Proba

In [45]:
def predict2(vid_pth):
    vidcap = cv2.VideoCapture(vid_pth)

    is_success, image = vidcap.read()
    frame_number = 0

    freq_dict = dict()

    probas_array = []

    while is_success:
        if frame_number%20 == 0:
            img = image[..., ::-1] / 255.
            img = np.expand_dims(cv2.resize(img, (256, 256)), axis=0)
            probas = model.predict(img, verbose=0)
            probas_array.append(probas)
            pred = probas.argmax()
            if pred in freq_dict:
                freq_dict[pred] += 1
            else:
                freq_dict[pred] = 1
        is_success, image = vidcap.read()
        frame_number += 1

    probas_array = np.array(probas_array)

    labels_dict = {0: "Rub both wrists in rotating manner",
                1: "Rub your palms together",
                2: "Rub the back of your fingers and hands",
                3: "Rub your hands by interlocking your fingers",
                4: "Interlock fingers and rub the back of fingers of both hands",
                5: "Rub the area between index finger and thumb",
                6: "Rub fingertips on palm of both hands in circular manner"}

    # max_freq_label = max(freq_dict.items(), key=lambda x: x[1])[0]
    max_mean_proba_label = probas_array.mean(axis=0).argmax()

    # return f"{labels_dict[max_freq_label]}"
    return f"{max_mean_proba_label}"

In [46]:
incorrect_preds2 = []
test_videos_dir = "/home/vikas/Projects/freelancing/research/dataset/videos/test"
for folder in os.listdir(test_videos_dir):
    for file in os.listdir(f"{test_videos_dir}/{folder}"):
        if folder != predict2(f"{test_videos_dir}/{folder}/{file}"):
            incorrect_preds2.append(f"{test_videos_dir}/{folder}/{file}")

In [47]:
len(incorrect_preds2)

22

In [48]:
incorrect_preds_dict2 = {}
for path in incorrect_preds2:
    incorrect_label = path.split("/")[-2]
    if incorrect_label in incorrect_preds_dict2:
        incorrect_preds_dict2[incorrect_label].append(path)
    else:
        incorrect_preds_dict2[incorrect_label] = [path]

In [50]:
incorrect_preds_dict2["0"]

['/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_019_A_11_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_015_A_12_G_04.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_020_A_11_G_05.mp4',
 '/home/vikas/Projects/freelancing/research/dataset/videos/test/0/HandWash_018_A_11_G_05.mp4']

In [51]:
incorrect_preds_dict["0"]

NameError: name 'incorrect_preds_dict' is not defined

In [67]:
def evaluate_videos2(test_videos_path):
    num_total_videos = 0
    num_correctly_predicted = 0
    for folder in os.listdir(test_videos_path):
        for file in os.listdir(f"{test_videos_path}/{folder}"):
            if folder == predict2(f"{test_videos_path}/{folder}/{file}"):
                num_correctly_predicted += 1
            num_total_videos += 1
        print(f"{folder=} processed successfully!")

    return num_correctly_predicted/num_total_videos*100

In [68]:
evaluate_videos2("dataset/videos/test")

folder='4' processed successfully!
folder='1' processed successfully!
folder='0' processed successfully!
folder='3' processed successfully!
folder='5' processed successfully!
folder='6' processed successfully!
folder='2' processed successfully!


73.17073170731707

In [74]:
def evaluate_videos(test_videos_path):
    num_total_videos = 0
    num_correctly_predicted = 0
    for folder in os.listdir(test_videos_path):
        for file in os.listdir(f"{test_videos_path}/{folder}"):
            if folder == predict(f"{test_videos_path}/{folder}/{file}"):
                num_correctly_predicted += 1
            num_total_videos += 1
        print(f"{folder=} processed successfully!")

    return num_correctly_predicted/num_total_videos*100

In [80]:
evaluate_videos("dataset/videos/test")

folder='4' processed successfully!
folder='1' processed successfully!
folder='0' processed successfully!
folder='3' processed successfully!
folder='5' processed successfully!
folder='6' processed successfully!
folder='2' processed successfully!


75.60975609756098

In [63]:
cnt = 0
for folder in os.listdir("dataset/videos/test"):
    cnt += len(os.listdir(f"dataset/videos/test/{folder}"))

In [64]:
cnt

82

In [76]:
82*90/100

73.8

In [77]:
74/82*100

90.2439024390244

In [78]:
82-74

8